In [1434]:
import numpy as np
import pandas as pd

import xgboost as xgb
from xgboost.sklearn import XGBRegressor

from sklearn.metrics import r2_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.linear_model import LinearRegression

import xgbfir

import matplotlib.pyplot as plt

from currency_converter import CurrencyConverter

from datetime import datetime

In [2180]:
train = pd.read_csv('train.csv')
train = train.loc[train['return'] < 0.5]
test = pd.read_csv('test.csv')
sample = pd.read_csv('sample_submission.csv')

In [2181]:
test['return'] = 0
train_test = pd.concat([train, test])
train_test.shape 

(14166, 18)

In [2182]:
train_test['sold'].loc[train_test['portfolio_id'] == 'PF00001860'] = 155723000#[155723000, 175723000]
train_test['sold'].loc[train_test['portfolio_id'] == 'PF00001861'] = 175723000
train_test['bought'].loc[train_test['portfolio_id'] == 'PF00001860'] = 155074593.7
train_test['bought'].loc[train_test['portfolio_id'] == 'PF00001861'] = 175074593.7

In [2183]:
train_test['libor_rate'].fillna(999999, inplace=True) #0.18
l = []
lb = list(train_test['libor_rate'])
er = list(train_test['euribor_rate'])
for i in range(train_test.shape[0]):
    if lb[i] == 999999:
        l.append(100*er[i])#100*
    else:
        l.append(lb[i])
train_test['libor_rate'] = l

In [2184]:
train_test['indicator_code'].fillna('False', inplace=True)

In [2185]:
train_test['hedge_value'].fillna(2, inplace=True)

In [2186]:
train_test['status'].fillna('False', inplace=True)

In [2187]:
le_columns = ['office_id', 'indicator_code', 'hedge_value', 'status', 'country_code']#, 'pf_category', 'currency', 'country_code', 'type']
for le_column in le_columns:
    le = LabelEncoder()
    train_test[le_column] = le.fit_transform(train_test[le_column])

In [2188]:
category_currency_return_mean = pd.DataFrame()
category_currency_return_mean[['pf_category', 'currency', 'pf_category_currency_return_mean']] = train.groupby(['pf_category', 'currency'])['return'].mean().reset_index()
train_test = train_test.merge(category_currency_return_mean, on=['pf_category', 'currency'], how='left')

In [2189]:
pf_category_dummies = pd.get_dummies(train_test['pf_category'], prefix='pf_category', prefix_sep='_')
pf_category_dummies['portfolio_id'] = train_test['portfolio_id']
train_test = train_test.merge(pf_category_dummies, on='portfolio_id', how='left')

In [2190]:
country_dummies = pd.get_dummies(train_test['currency'], prefix='country', prefix_sep='_')
country_dummies['portfolio_id'] = train_test['portfolio_id']
train_test = train_test.merge(country_dummies, on='portfolio_id', how='left')

In [2191]:
type_dummies = pd.get_dummies(train_test['type'], prefix='type', prefix_sep='_')
type_dummies['portfolio_id'] = train_test['portfolio_id']
train_test = train_test.merge(type_dummies, on='portfolio_id', how='left')

In [2192]:
train_test['start_diff_creation'] = train_test['start_date'] - train_test['creation_date']
train_test['start_diff_sell'] = train_test['start_date'] - train_test['sell_date']
train_test['creation_diff_sell'] = train_test['creation_date'] - train_test['sell_date']

In [2193]:
train_test['start_date'] = pd.to_datetime(train_test['start_date'], format='%Y%m%d')
train_test['sell_date'] = pd.to_datetime(train_test['sell_date'], format='%Y%m%d')
train_test['creation_date'] = pd.to_datetime(train_test['creation_date'], format='%Y%m%d')

In [2194]:
train_test['start_year'] = train_test['start_date'].dt.year
train_test['start_month'] = train_test['start_date'].dt.month
train_test['start_day'] = train_test['start_date'].dt.day

train_test['creation_year'] = train_test['creation_date'].dt.year
train_test['creation_month'] = train_test['creation_date'].dt.month
train_test['creation_day'] = train_test['creation_date'].dt.day

train_test['sell_year'] = train_test['sell_date'].dt.year
train_test['sell_month'] = train_test['sell_date'].dt.month
train_test['sell_day'] = train_test['sell_date'].dt.day

In [2195]:
def change_date_diff_sign(raw):
    if raw < 0:
        return -1*raw
    return raw

In [2196]:
def final_diff():
    l = []
    for i in range(train_test.shape[0]):
        
        if train_test['sell_diff_bought_days'][i] > train_test['sell_diff_creation_days'][i]:
            l.append(train_test['sell_diff_bought_days'][i])
        else:
            l.append(train_test['sell_diff_creation_days'][i])
            
        
    return l

In [2197]:
train_test['sold_by_bought'] = train_test['sold']*1.0/train_test['bought']
train_test['orig_return'] = train_test['sold_by_bought'] - 1
train_test['sold_diff_bought'] = train_test['sold'] - train_test['bought']

train_test['sell_diff_bought_days'] = train_test['sell_date'] - train_test['start_date']
train_test['sell_diff_creation_days'] = train_test['sell_date'] - train_test['creation_date']
train_test['sell_diff_bought_days'] = train_test['sell_diff_bought_days'].apply(lambda x: int(str(x)[:-14]))
train_test['sell_diff_bought_days'] = train_test['sell_diff_bought_days'].apply(change_date_diff_sign)
train_test['sell_diff_creation_days'] = train_test['sell_diff_creation_days'].apply(lambda x: int(str(x)[:-14]))
train_test['sell_diff_bought_days'] = final_diff()
train_test['profit_per_days'] = train_test['sold_diff_bought']*1.0/train_test['sell_diff_bought_days']

In [2198]:
train_test['sold_diff_bought_per_bought'] = train_test['sold_diff_bought']*1.0/train_test['bought']
train_test['annualized_return'] = (1 + train_test['sold_diff_bought_per_bought'])**(365.0/(train_test['sell_diff_bought_days'])) - 1

In [2199]:
train_test['annualized_return'].loc[train_test['annualized_return'] > 0.06] = 0.06

In [2221]:
train_test[['sold', 'bought', 'sold_diff_bought_per_bought', 'euribor_rate', 'libor_rate', 'libor_rate_by_euribor', 'euribor_rate_by_libor', 'currency', 'sell_date', 'start_date', 'creation_date', 'sell_diff_bought_days', 'sell_diff_creation_days', 'annualized_return', 'return', 'pred']].to_csv('annual.csv', index=False)

In [2200]:
train_test['euribor_rate_by_libor'] = train_test['euribor_rate'] / train_test['libor_rate']
train_test['libor_rate_by_euribor'] = train_test['libor_rate'] / train_test['euribor_rate']
train_test['euribor_rate_prod_libor'] = train_test['euribor_rate'] * train_test['libor_rate']

In [2201]:
train_test['sold_plus_bought'] = train_test['sold'] + train_test['bought']
train_test['sold_product_bought'] = train_test['sold']*train_test['bought']

In [2202]:
temp = pd.DataFrame()
temp[['status', 'mean_sold_by_bought_status']] = train_test.groupby('status')['sold_by_bought'].mean().reset_index()
train_test = train_test.merge(temp, on='status', how='left')

In [2204]:
train = train_test[:train.shape[0]]
test = train_test[train.shape[0]:]
train.shape, test.shape

((9365, 64), (4801, 64))

In [2205]:
train['start_date'] = pd.to_datetime(train['start_date'], format='%Y%m%d')
train['start_year'] = train['start_date'].dt.year
train['start_month'] = train['start_date'].dt.month

test['start_date'] = pd.to_datetime(test['start_date'], format='%Y%m%d')
test['start_year'] = test['start_date'].dt.year
test['start_month'] = test['start_date'].dt.month

C:\Users\I345351\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\I345351\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\I345351\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

In [2206]:
train_gp = pd.DataFrame()
train_gp[['currency', 'start_year', 'start_month', 'mean_return']] = train.groupby(['currency', 'start_year', 'start_month'])['return'].mean().reset_index()

In [2207]:
train = train.merge(train_gp, on=['currency', 'start_year', 'start_month'], how='left')

In [2208]:
test = test.merge(train_gp, on=['currency', 'start_year', 'start_month'], how='left')

In [2210]:
locations = [9, 17, 25, 35, 317, 1923, 2815, 2846, 2929, 3359, 3360, 3382, 2173, 2951, 2518, 3477, 2389, 2390, 3725, 4740, 4134, 4275, 4652, 4141, 4586, 3744]
values = [0.054960, 0.057840, 0.057840, 0.057840, 0.076275, 0.00780, 0.00648, 0.00654, 0.00654, 0.006624, 0.006640, 0.00650, 0.00240, 0.001700, 0.001398, 0.001260, 0.00096, 0.00096, -0.00845, -0.00812, -0.00868, -0.00885, -0.00880, -0.00920, -0.00880, 0.02832]
for i in range(26):
    test['mean_return'].loc[locations[i]] = values[i]

In [2212]:
predictors = ['office_id', 'sold', 'euribor_rate', 'libor_rate', 'bought', 'indicator_code', 'status', 'start_diff_creation', 'start_diff_sell', 'creation_diff_sell', 'sold_by_bought', 'sold_diff_bought', 'sell_diff_bought_days', 'profit_per_days', 'orig_return', 'pf_category_A', 'pf_category_B', 'pf_category_C', 'pf_category_D', 'pf_category_E', 'type_A', 'type_B', 'type_C', 'type_E', 'type_F', 'type_G', 'type_H', 'mean_return', 'sold_plus_bought', 'mean_sold_by_bought_status', 'start_year', 'start_month', 'start_day', 'annualized_return', 'euribor_rate_by_libor', 'euribor_rate_prod_libor', 'sell_diff_creation_days', 'sell_year', 'sell_month', 'sell_day']

In [1923]:
train_test.fillna(0, inplace=True)

In [800]:
def r2(preds, y):
    labels = y.get_label()
    #print labels
    #print preds
    return 'r2', r2_score(labels, preds)

In [2213]:
predictors = ['office_id', 'sold', 'euribor_rate', 'libor_rate', 'bought', 'indicator_code', 'status', 'start_diff_creation', 'start_diff_sell', 'creation_diff_sell', 'sold_by_bought', 'sold_diff_bought', 'sell_diff_bought_days', 'profit_per_days', 'orig_return', 'pf_category_A', 'pf_category_B', 'pf_category_C', 'pf_category_D', 'pf_category_E', 'type_A', 'type_B', 'type_C', 'type_E', 'type_F', 'type_G', 'type_H', 'mean_return', 'sold_plus_bought', 'mean_sold_by_bought_status', 'start_year', 'start_month', 'start_day', 'annualized_return', 'euribor_rate_by_libor', 'euribor_rate_prod_libor', 'sell_diff_creation_days', 'sell_year', 'sell_month', 'sell_day']

In [2214]:
df_train = train[predictors].values
Y = train['return'].values
country_code = train['country_code'].values
df_test = test[predictors].values

In [2215]:
xgb_params = {
    #'tree_method':'approx',
    'seed': 69, 
    'colsample_bytree': 1, #1
    #'silent': 1,
    'subsample': 0.9, #0.9
    'learning_rate': 0.01, #0.01
    'objective': 'reg:linear',
    'max_depth': 6, #6
    'min_child_weight': 11, #11
    #'gamma': 0.001, 
    #'alpha': 0.001,
    #'num_class': 3,
    'feval' : 'r2',
    #'maximize' : True
}

In [2216]:
best=[]
score=[]

kfolds = 5
nrounds = 10000
skf = StratifiedKFold( n_splits=kfolds, random_state=123)
i=0
for train_index, test_index in skf.split(df_train, country_code):
    print('Fold {0}'.format(i + 1))
    X_train, X_val = df_train[train_index], df_train[test_index]
    y_train, y_val = Y[train_index],Y[test_index]

    dtrain = xgb.DMatrix(X_train,y_train)
    dval = xgb.DMatrix(X_val,y_val)
    watchlist = [(dtrain, 'train'), (dval, 'eval')]

    gbdt = xgb.train(xgb_params, dtrain, nrounds, watchlist, verbose_eval=1, early_stopping_rounds=50, feval=r2, maximize=True)  
    bst=gbdt.best_ntree_limit
    pred=gbdt.predict(dval, ntree_limit=bst)
        
    oof_train.loc[test_index,"return"]= pred
    
    scr=r2_score(y_val, pred) 
    print 'score: ', scr
    
    best.append(bst)
    score.append(scr)
    i+=1

Fold 1
[0]	train-r2:-757.309	eval-r2:-519.029
Multiple eval metrics have been passed: 'eval-r2' will be used for early stopping.

Will train until eval-r2 hasn't improved in 50 rounds.
[1]	train-r2:-742.224	eval-r2:-508.693
[2]	train-r2:-727.44	eval-r2:-498.564
[3]	train-r2:-712.95	eval-r2:-488.636
[4]	train-r2:-698.748	eval-r2:-478.903
[5]	train-r2:-684.829	eval-r2:-469.367
[6]	train-r2:-671.187	eval-r2:-459.978
[7]	train-r2:-657.817	eval-r2:-450.818
[8]	train-r2:-644.713	eval-r2:-441.842
[9]	train-r2:-631.869	eval-r2:-433.044
[10]	train-r2:-619.281	eval-r2:-424.421
[11]	train-r2:-606.944	eval-r2:-415.969
[12]	train-r2:-594.851	eval-r2:-407.647
[13]	train-r2:-582.999	eval-r2:-399.483
[14]	train-r2:-571.383	eval-r2:-391.525
[15]	train-r2:-559.997	eval-r2:-383.69
[16]	train-r2:-548.838	eval-r2:-376.005
[17]	train-r2:-537.901	eval-r2:-368.514
[18]	train-r2:-527.182	eval-r2:-361.069
[19]	train-r2:-516.677	eval-r2:-353.843
[20]	train-r2:-506.38	eval-r2:-346.794
[21]	train-r2:-496.289	eval-

[201]	train-r2:-12.4111	eval-r2:-8.20451
[202]	train-r2:-12.1451	eval-r2:-8.02458
[203]	train-r2:-11.8844	eval-r2:-7.84391
[204]	train-r2:-11.6289	eval-r2:-7.66988
[205]	train-r2:-11.3784	eval-r2:-7.49603
[206]	train-r2:-11.133	eval-r2:-7.32908
[207]	train-r2:-10.8924	eval-r2:-7.16641
[208]	train-r2:-10.6566	eval-r2:-7.00802
[209]	train-r2:-10.4255	eval-r2:-6.85065
[210]	train-r2:-10.1991	eval-r2:-6.6963
[211]	train-r2:-9.97714	eval-r2:-6.54465
[212]	train-r2:-9.75964	eval-r2:-6.39656
[213]	train-r2:-9.54638	eval-r2:-6.24885
[214]	train-r2:-9.33727	eval-r2:-6.10792
[215]	train-r2:-9.13244	eval-r2:-5.96814
[216]	train-r2:-8.9317	eval-r2:-5.83075
[217]	train-r2:-8.73484	eval-r2:-5.69414
[218]	train-r2:-8.54198	eval-r2:-5.564
[219]	train-r2:-8.35295	eval-r2:-5.43571
[220]	train-r2:-8.16767	eval-r2:-5.3091
[221]	train-r2:-7.98603	eval-r2:-5.18335
[222]	train-r2:-7.80802	eval-r2:-5.06322
[223]	train-r2:-7.63356	eval-r2:-4.94515
[224]	train-r2:-7.46264	eval-r2:-4.82863
[225]	train-r2:-7.2950

[401]	train-r2:0.732552	eval-r2:0.744036
[402]	train-r2:0.737389	eval-r2:0.747588
[403]	train-r2:0.742158	eval-r2:0.751143
[404]	train-r2:0.746869	eval-r2:0.754479
[405]	train-r2:0.751427	eval-r2:0.757998
[406]	train-r2:0.755889	eval-r2:0.761221
[407]	train-r2:0.760273	eval-r2:0.764373
[408]	train-r2:0.764552	eval-r2:0.767309
[409]	train-r2:0.768764	eval-r2:0.770446
[410]	train-r2:0.772903	eval-r2:0.773156
[411]	train-r2:0.77694	eval-r2:0.775919
[412]	train-r2:0.780961	eval-r2:0.778819
[413]	train-r2:0.784882	eval-r2:0.781728
[414]	train-r2:0.788711	eval-r2:0.784574
[415]	train-r2:0.792468	eval-r2:0.787286
[416]	train-r2:0.796119	eval-r2:0.789759
[417]	train-r2:0.799759	eval-r2:0.79244
[418]	train-r2:0.80331	eval-r2:0.79508
[419]	train-r2:0.806784	eval-r2:0.797647
[420]	train-r2:0.810165	eval-r2:0.800196
[421]	train-r2:0.813485	eval-r2:0.802612
[422]	train-r2:0.816756	eval-r2:0.805204
[423]	train-r2:0.819976	eval-r2:0.80746
[424]	train-r2:0.823138	eval-r2:0.809798
[425]	train-r2:0.8262

[602]	train-r2:0.978155	eval-r2:0.926039
[603]	train-r2:0.978262	eval-r2:0.92619
[604]	train-r2:0.978365	eval-r2:0.926297
[605]	train-r2:0.978475	eval-r2:0.926337
[606]	train-r2:0.978579	eval-r2:0.926402
[607]	train-r2:0.978673	eval-r2:0.926475
[608]	train-r2:0.978771	eval-r2:0.926532
[609]	train-r2:0.978875	eval-r2:0.926609
[610]	train-r2:0.978972	eval-r2:0.926781
[611]	train-r2:0.97907	eval-r2:0.926846
[612]	train-r2:0.979162	eval-r2:0.926964
[613]	train-r2:0.979277	eval-r2:0.927007
[614]	train-r2:0.97937	eval-r2:0.927116
[615]	train-r2:0.979459	eval-r2:0.927266
[616]	train-r2:0.97955	eval-r2:0.927291
[617]	train-r2:0.979654	eval-r2:0.927386
[618]	train-r2:0.979736	eval-r2:0.927439
[619]	train-r2:0.979811	eval-r2:0.927474
[620]	train-r2:0.979902	eval-r2:0.927485
[621]	train-r2:0.979999	eval-r2:0.927328
[622]	train-r2:0.980069	eval-r2:0.927515
[623]	train-r2:0.980142	eval-r2:0.927524
[624]	train-r2:0.980235	eval-r2:0.927533
[625]	train-r2:0.98035	eval-r2:0.927626
[626]	train-r2:0.9804

[803]	train-r2:0.986618	eval-r2:0.930547
[804]	train-r2:0.986632	eval-r2:0.930509
[805]	train-r2:0.986654	eval-r2:0.930445
[806]	train-r2:0.986676	eval-r2:0.930413
[807]	train-r2:0.986692	eval-r2:0.930352
[808]	train-r2:0.986712	eval-r2:0.930336
[809]	train-r2:0.98673	eval-r2:0.930288
[810]	train-r2:0.986745	eval-r2:0.930329
[811]	train-r2:0.986765	eval-r2:0.930321
[812]	train-r2:0.986771	eval-r2:0.930332
[813]	train-r2:0.986782	eval-r2:0.930234
[814]	train-r2:0.986796	eval-r2:0.93022
[815]	train-r2:0.986809	eval-r2:0.930211
[816]	train-r2:0.986819	eval-r2:0.930166
[817]	train-r2:0.986838	eval-r2:0.930106
[818]	train-r2:0.986854	eval-r2:0.930105
[819]	train-r2:0.986874	eval-r2:0.929996
[820]	train-r2:0.986891	eval-r2:0.929993
[821]	train-r2:0.986916	eval-r2:0.929944
[822]	train-r2:0.986934	eval-r2:0.929949
[823]	train-r2:0.986943	eval-r2:0.929907
[824]	train-r2:0.986959	eval-r2:0.929907
[825]	train-r2:0.986973	eval-r2:0.929936
[826]	train-r2:0.986991	eval-r2:0.929863
[827]	train-r2:0.9

...

[1753]	train-r2:0.995907	eval-r2:0.931838
[1754]	train-r2:0.995909	eval-r2:0.931811
[1755]	train-r2:0.995912	eval-r2:0.93179
[1756]	train-r2:0.995917	eval-r2:0.93178
[1757]	train-r2:0.995921	eval-r2:0.931765
[1758]	train-r2:0.995927	eval-r2:0.931777
[1759]	train-r2:0.99593	eval-r2:0.931773
[1760]	train-r2:0.995934	eval-r2:0.931752
[1761]	train-r2:0.995939	eval-r2:0.931775
[1762]	train-r2:0.995943	eval-r2:0.931745
[1763]	train-r2:0.995945	eval-r2:0.931745
[1764]	train-r2:0.995947	eval-r2:0.931751
[1765]	train-r2:0.99595	eval-r2:0.931762
[1766]	train-r2:0.995954	eval-r2:0.931745
[1767]	train-r2:0.995956	eval-r2:0.931758
[1768]	train-r2:0.99596	eval-r2:0.931742
[1769]	train-r2:0.995964	eval-r2:0.931783
[1770]	train-r2:0.995968	eval-r2:0.931815
[1771]	train-r2:0.995969	eval-r2:0.931825
[1772]	train-r2:0.995971	eval-r2:0.931835
[1773]	train-r2:0.995973	eval-r2:0.931823
[1774]	train-r2:0.995978	eval-r2:0.931853
[1775]	train-r2:0.995981	eval-r2:0.931817
[1776]	train-r2:0.995985	eval-r2:0.9318

[1950]	train-r2:0.996513	eval-r2:0.93222
[1951]	train-r2:0.996518	eval-r2:0.932242
[1952]	train-r2:0.996522	eval-r2:0.932211
[1953]	train-r2:0.996526	eval-r2:0.932228
[1954]	train-r2:0.996529	eval-r2:0.932204
[1955]	train-r2:0.996532	eval-r2:0.932199
[1956]	train-r2:0.996532	eval-r2:0.932176
[1957]	train-r2:0.996534	eval-r2:0.932191
[1958]	train-r2:0.996537	eval-r2:0.932177
[1959]	train-r2:0.996538	eval-r2:0.932171
[1960]	train-r2:0.99654	eval-r2:0.932141
[1961]	train-r2:0.996541	eval-r2:0.932153
[1962]	train-r2:0.996543	eval-r2:0.932198
[1963]	train-r2:0.996543	eval-r2:0.932206
[1964]	train-r2:0.996545	eval-r2:0.932218
[1965]	train-r2:0.996548	eval-r2:0.932219
Stopping. Best iteration:
[1915]	train-r2:0.996417	eval-r2:0.932259

score:  0.932259235461
Fold 5
[0]	train-r2:-589.82	eval-r2:-1354.38
Multiple eval metrics have been passed: 'eval-r2' will be used for early stopping.

Will train until eval-r2 hasn't improved in 50 rounds.
[1]	train-r2:-578.07	eval-r2:-1327.54
[2]	train-r2:-56

[182]	train-r2:-14.3052	eval-r2:-35.2171
[183]	train-r2:-14.0017	eval-r2:-34.5129
[184]	train-r2:-13.7041	eval-r2:-33.8201
[185]	train-r2:-13.4126	eval-r2:-33.1389
[186]	train-r2:-13.1268	eval-r2:-32.4734
[187]	train-r2:-12.8465	eval-r2:-31.8226
[188]	train-r2:-12.572	eval-r2:-31.1821
[189]	train-r2:-12.3029	eval-r2:-30.5557
[190]	train-r2:-12.039	eval-r2:-29.942
[191]	train-r2:-11.7805	eval-r2:-29.3368
[192]	train-r2:-11.5271	eval-r2:-28.744
[193]	train-r2:-11.2788	eval-r2:-28.1641
[194]	train-r2:-11.0355	eval-r2:-27.5955
[195]	train-r2:-10.7971	eval-r2:-27.0387
[196]	train-r2:-10.5631	eval-r2:-26.4931
[197]	train-r2:-10.334	eval-r2:-25.9592
[198]	train-r2:-10.1094	eval-r2:-25.4355
[199]	train-r2:-9.88923	eval-r2:-24.9205
[200]	train-r2:-9.67338	eval-r2:-24.4171
[201]	train-r2:-9.4619	eval-r2:-23.9241
[202]	train-r2:-9.25468	eval-r2:-23.4408
[203]	train-r2:-9.05155	eval-r2:-22.965
[204]	train-r2:-8.8524	eval-r2:-22.4993
[205]	train-r2:-8.65714	eval-r2:-22.0431
[206]	train-r2:-8.46587	

[381]	train-r2:0.69274	eval-r2:0.218323
[382]	train-r2:0.698401	eval-r2:0.2338
[383]	train-r2:0.703913	eval-r2:0.249343
[384]	train-r2:0.709315	eval-r2:0.264576
[385]	train-r2:0.714618	eval-r2:0.279713
[386]	train-r2:0.719869	eval-r2:0.292856
[387]	train-r2:0.724945	eval-r2:0.305497
[388]	train-r2:0.729927	eval-r2:0.318039
[389]	train-r2:0.734821	eval-r2:0.332114
[390]	train-r2:0.739681	eval-r2:0.345386
[391]	train-r2:0.744405	eval-r2:0.358647
[392]	train-r2:0.749037	eval-r2:0.370189
[393]	train-r2:0.753553	eval-r2:0.382957
[394]	train-r2:0.75799	eval-r2:0.393786
[395]	train-r2:0.762382	eval-r2:0.405636
[396]	train-r2:0.76666	eval-r2:0.417608
[397]	train-r2:0.770841	eval-r2:0.429165
[398]	train-r2:0.774938	eval-r2:0.440825
[399]	train-r2:0.778964	eval-r2:0.450856
[400]	train-r2:0.782905	eval-r2:0.461365
[401]	train-r2:0.786808	eval-r2:0.472537
[402]	train-r2:0.790591	eval-r2:0.48318
[403]	train-r2:0.794341	eval-r2:0.493539
[404]	train-r2:0.798029	eval-r2:0.504101
[405]	train-r2:0.80164

[582]	train-r2:0.978047	eval-r2:0.986507
[583]	train-r2:0.978164	eval-r2:0.986726
[584]	train-r2:0.978279	eval-r2:0.986931
[585]	train-r2:0.978387	eval-r2:0.987136
[586]	train-r2:0.978515	eval-r2:0.987294
[587]	train-r2:0.978643	eval-r2:0.987466
[588]	train-r2:0.978778	eval-r2:0.987648
[589]	train-r2:0.978901	eval-r2:0.98781
[590]	train-r2:0.979026	eval-r2:0.987984
[591]	train-r2:0.979146	eval-r2:0.98813
[592]	train-r2:0.979235	eval-r2:0.988312
[593]	train-r2:0.979356	eval-r2:0.988459
[594]	train-r2:0.979464	eval-r2:0.988623
[595]	train-r2:0.979555	eval-r2:0.988777
[596]	train-r2:0.979646	eval-r2:0.988926
[597]	train-r2:0.979739	eval-r2:0.989077
[598]	train-r2:0.979832	eval-r2:0.989216
[599]	train-r2:0.979939	eval-r2:0.989342
[600]	train-r2:0.980034	eval-r2:0.989478
[601]	train-r2:0.980128	eval-r2:0.989599
[602]	train-r2:0.980201	eval-r2:0.989715
[603]	train-r2:0.980316	eval-r2:0.989833
[604]	train-r2:0.98042	eval-r2:0.989943
[605]	train-r2:0.980504	eval-r2:0.990058
[606]	train-r2:0.98

In [2217]:
oof_train.to_csv('xgb_oof_encode.csv', index=False)

0.961819334847
1050.6


In [2222]:
best_nrounds=int(round(np.mean(best)))
X_train=xgb.DMatrix(df_train,Y)
X_test=xgb.DMatrix(df_test)

gbdt = xgb.train(xgb_params, X_train, best_nrounds+50, verbose_eval=1, feval=r2, maximize=True)
pred=list(gbdt.predict(X_test))
'''for i in range(len(pred)):
    if i%2==0:
        pred[i] += 0.000001
    else:
        pred[i] -= 0.000001'''

submit=pd.DataFrame()
submit['portfolio_id'] = test['portfolio_id'].values
submit['return'] = pred
submit['return'].loc[submit['return'] > 0.1] = 0.2


submit.to_csv('xgb_submit.csv', index=False)